In [3]:
#content-based music recommender system
#Importing libraries
pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import numpy as np
import pandas as pd

In [6]:
from typing import List, Dict


In [8]:
#we have already used in TF-IDF score before when performing Twitter sentiment analysis
#Likewise,we are going to use Tfidfvectorizer from the Scikit-learn package again.
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Dataset

so imagine that we have the following dataset.

This dataset contains name,artist, and lyrics for 57650 songs in English.

This data has been acquired from LyricsFreak througn scraping.

In [10]:
songs = pd.read_csv("C:\\Users\\nagar\\Downloads\\songsdata.csv")

In [12]:

songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,People Need Love,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


Because of the dataset being so big,we are going to resample only 5000 random songs

In [13]:
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

We can notice also the presence of \n the text,so we are going to remove it

In [14]:
songs['text'] = songs['text'].str.replace(r'\n', '')

After that,we use TF-IDF vectorizer that calculates the TF-IDF score for each song lyric,word by word

Here,we pay particular attention to the arguments we can specify.

In [15]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [16]:
lyrics_matrix = tfidf.fit_transform(songs['text'])

How do we use this matrix for a recommendation?

We now need to calculate the similarity of one lyric to another .We are going to usr cosine similarity.

We want to calculate the cosine similarity of each item with every other item in the dataset.So we just pass the lyrics_matrix as agrument

In [17]:
cosine_similarities = cosine_similarity(lyrics_matrix) 

Once we get the similarites,we'll store in a dictionary the names  of the 50 most similar songs for each sing in our dataset

In [18]:
similarities = {}

In [19]:
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs. 
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

We can use that similarity scores to access the most similar items and give a recommendation

For that, we'll define our content based recommender class:

In [20]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
        
    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

In [21]:
#now,instantiate class
recommedations = ContentBasedRecommender(similarities)

Then,we are ready to pick a song from the dataset and make arecommendation

In [22]:
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4 
}

In [23]:
recommedations.recommend(recommendation)

The 4 recommended songs for Granny are:
Number 1:
Running Wild by Judas Priest with 0.357 similarity score
--------------------
Number 2:
Wild America by Iggy Pop with 0.272 similarity score
--------------------
Number 3:
Young Volcanoes by Fall Out Boy with 0.257 similarity score
--------------------
Number 4:
Life Is Wild by Clash with 0.251 similarity score
--------------------


In [24]:
#And we pick another random somg and recommend song
recommendation2 = {
    "song": songs['song'].iloc[120],
    "number_songs": 4 
}

In [25]:
recommedations.recommend(recommendation2)

The 4 recommended songs for Clinging To A Saving Hand are:
Number 1:
Why We Sing by Kirk Franklin with 0.304 similarity score
--------------------
Number 2:
Bombollini by Chris Rea with 0.246 similarity score
--------------------
Number 3:
Maybe Tomorrow by UB40 with 0.234 similarity score
--------------------
Number 4:
Joy To The World by Faith Hill with 0.219 similarity score
--------------------
